In [ ]:
WIP final

In [ ]:
%%sql
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('WIP', p.id) AS patient_id,
    p.id AS src_patient_id,
    -- 'Unknown' AS patient_full_name,

    CASE
        WHEN p.date_of_birth IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    CASE
        WHEN p.date_of_birth IS NOT NULL THEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE())
        ELSE 9999
    END AS patient_current_age,

    'Unknown' AS patient_gender,
    'Unknown' AS patient_gender_code_conformed,

    1 AS z_record_is_active,
    'WIP000' AS z_src_system_instance,
    'WIP' AS z_src_system_id,

    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP() AS z_record_created_date_time,
    CURRENT_USER() AS z_record_created_by_user,
    CURRENT_TIMESTAMP() AS z_record_modified_date_time,
    CURRENT_USER() AS z_record_modified_by_user

FROM silver_wip_person p;
रन केल्यावर लगेच हे 4 verification checks कर
1) Source मध्ये DOB null किती?
%%sql
SELECT COUNT(*) AS wip_dob_null
FROM silver_wip_person
WHERE date_of_birth IS NULL;
2) Mock table मध्ये WIP rows किती insert झाले?
%%sql
SELECT COUNT(*) AS wip_rows_in_mock
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP';
3) 9999 count = DOB null count आहे का? (हे key check)
%%sql
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP';
4) Quick sample (age_range/age sanity)
%%sql
SELECT patient_id, src_patient_id, age_range, patient_current_age
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP'
LIMIT 20;

cf-- final 

In [ ]:
-- 0) (One-time per rerun) delete CF rows from mock table
DELETE FROM silver_patient_mock
WHERE z_src_system_id = 'CF';


-- 1) Insert CF rows (final, business-safe)
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('CF', CAST(p.customer_id AS STRING))                                  AS patient_id,
    CAST(p.customer_id AS STRING)                                               AS src_patient_id,

    /* age_range (business-safe) */
    CASE
        WHEN dob_dt IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) > 120 THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END                                                                           AS age_range,

    /* patient_current_age (business-safe) */
    CASE
        WHEN dob_dt IS NULL THEN 9999
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) > 120 THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE())
    END                                                                           AS patient_current_age,

    p.customer_gender                                                             AS patient_gender,

    CASE
        WHEN p.customer_gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
        WHEN p.customer_gender IN ('M','Male','Male (including trans man)')       THEN 'Male'
        WHEN p.customer_gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END                                                                           AS patient_gender_code_conformed,

    COALESCE(p.customer_status, 0)                                                AS z_record_is_active,
    'CF000'                                                                       AS z_src_system_instance,
    'CF'                                                                          AS z_src_system_id,

    'Unknown'                                                                     AS z_src_system_created_date_time,
    'Unknown'                                                                     AS z_src_system_created_by_user,
    'Unknown'                                                                     AS z_src_system_modified_date_time,
    'Unknown'                                                                     AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP()                                                           AS z_record_created_date_time,
    CURRENT_USER()                                                                AS z_record_created_by_user,
    CURRENT_TIMESTAMP()                                                           AS z_record_modified_date_time,
    CURRENT_USER()                                                                AS z_record_modified_by_user
FROM (
    SELECT
        p.*,

        /* Safe DOB parsing -> DATE */
        TO_DATE(
            COALESCE(
                try_to_timestamp(NULLIF(TRIM(p.customer_date_of_birth), ''), 'yyyy-MM-dd HH:mm:ss'),
                try_to_timestamp(NULLIF(TRIM(p.customer_date_of_birth), ''), 'yyyy-MM-dd')
            )
        ) AS dob_dt

    FROM silver_cf_tblcustomer p
) p;


In [ ]:
-- A) CF rows inserted count
SELECT COUNT(*) AS cf_rows
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

-- B) How many got 9999 (invalid / missing / crazy DOB)
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

-- C) Show the records that became 9999 (to inspect)
SELECT src_patient_id, patient_id, patient_current_age, age_range
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
  AND patient_current_age = 9999
LIMIT 50;


TM3 final

In [ ]:
-- Rerun safe
DELETE FROM silver_patient_mock
WHERE z_src_system_id = 'TM3';

INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('TM3', p.client_reference, p.client_pk) AS patient_id,
    CAST(p.client_pk AS STRING)                    AS src_patient_id,

    /* age_range */
    CASE
        WHEN p.client_date_of_birth IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) > 120 THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    /* patient_current_age */
    CASE
        WHEN p.client_date_of_birth IS NULL THEN 9999
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) > 120 THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE())
    END AS patient_current_age,

    p.client_gender AS patient_gender,

    CASE
        WHEN UPPER(TRIM(p.client_gender)) IN ('F','FEMALE','FEMALE (INCLUDING TRANS WOMAN)') THEN 'Female'
        WHEN UPPER(TRIM(p.client_gender)) IN ('M','MALE','MALE (INCLUDING TRANS MAN)') THEN 'Male'
        WHEN UPPER(TRIM(p.client_gender)) IN ('OTHER','NON-BINARY','OTHER GENDER NOT LISTED') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    1 AS z_record_is_active,
    'TM3000' AS z_src_system_instance,
    'TM3' AS z_src_system_id,

    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP() AS z_record_created_date_time,
    CURRENT_USER()      AS z_record_created_by_user,
    CURRENT_TIMESTAMP() AS z_record_modified_date_time,
    CURRENT_USER()      AS z_record_modified_by_user

FROM silver_tm3_dim_clients p;


In [ ]:
-- Row count match
SELECT COUNT(*) FROM silver_tm3_dim_clients;

SELECT COUNT(*)
FROM silver_patient_mock
WHERE z_src_system_id = 'TM3';


-- Null age check
SELECT COUNT(*)
FROM silver_patient_mock
WHERE z_src_system_id = 'TM3'
AND patient_current_age IS NULL;


-- 9999 count
SELECT COUNT(*)
FROM silver_patient_mock
WHERE z_src_system_id = 'TM3'
AND patient_current_age = 9999;


In [ ]:
Completed development and validation of Silver Patient build for all assigned source systems (IAPT, WIP, CF, TM3).

Work included:
Implemented updated Silver → silver_patient_mock logic for IAPT, WIP, CF and TM3
Insert logic aligned with agreed UDM structure

System-specific handling:

IAPT – standard DOB logic

WIP – date_of_birth handling

CF – string DOB safe parsing + bad record handling

TM3 – timestamp DOB direct calculation

Implemented business-safe age logic (null/invalid DOB → 9999)

Applied consistent gender conformed mapping

Performed full row count reconciliation per source

Executed null and edge-case validations

Resolved Spark parsing error (cross-version datetime parser issue)

All checks completed successfully.

In [ ]:
sone- final

In [ ]:
INSERT INTO TABLE silver_patient_mock
SELECT
  CONCAT('SONE',p.id_organisation_source,p.id) AS patient_id,
  p.id AS src_patient_id,
  CASE
    WHEN p.dob_dt IS NULL THEN 'Unknown'
    WHEN TIMESTAMPDIFF(YEAR,p.dob_dt,CURRENT_DATE()) < 18 THEN 'Under 18'
    WHEN TIMESTAMPDIFF(YEAR,p.dob_dt,CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
    WHEN TIMESTAMPDIFF(YEAR,p.dob_dt,CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
    WHEN TIMESTAMPDIFF(YEAR,p.dob_dt,CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
    WHEN TIMESTAMPDIFF(YEAR,p.dob_dt,CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
    WHEN TIMESTAMPDIFF(YEAR,p.dob_dt,CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
    ELSE '65 and over'
  END AS age_range,
  CASE
    WHEN p.dob_dt IS NULL THEN 9999
    ELSE TIMESTAMPDIFF(YEAR,p.dob_dt,CURRENT_DATE())
  END AS patient_current_age,
  p.gender AS patient_gender,
  CASE
    WHEN p.gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
    WHEN p.gender IN ('M','Male','Male (including trans man)') THEN 'Male'
    WHEN p.gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
    ELSE 'Unknown'
  END AS patient_gender_code_conformed,
  1 AS z_record_is_active,
  CONCAT('SONE',p.id_organisation_source) AS z_src_system_instance,
  'SONE' AS z_src_system_id,
  'Unknown' AS z_src_system_created_date_time,
  'Unknown' AS z_src_system_created_by_user,
  'Unknown' AS z_src_system_modified_date_time,
  'Unknown' AS z_src_system_modified_by_user,
  CURRENT_TIMESTAMP() AS z_record_created_date_time,
  CURRENT_USER() AS z_record_created_by_user,
  CURRENT_TIMESTAMP() AS z_record_modified_date_time,
  CURRENT_USER() AS z_record_modified_by_user
FROM (
  SELECT
    sp.*,
    COALESCE(
      TRY_TO_DATE(sp.date_birth,'dd MMM yyyy HH:mm'),
      TRY_TO_DATE(sp.date_birth,'dd MMM yyyy')
    ) AS dob_dt
  FROM silver_sone_srpatient sp
) p;


In [ ]:
drj/mpb final

In [ ]:
-- DRJ / MPB (same) -> insert into the new consolidated table
INSERT INTO TABLE silver_patient_mock
SELECT
  CONCAT('MPB',u.group_id) AS patient_id,
  u.group_id AS src_patient_id,
  'Unknown' AS patient_full_name,
  CASE
    WHEN u.birthday IS NULL THEN 'Unknown'
    WHEN TIMESTAMPDIFF(YEAR,u.birthday,CURRENT_DATE()) < 18 THEN 'Under 18'
    WHEN TIMESTAMPDIFF(YEAR,u.birthday,CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
    WHEN TIMESTAMPDIFF(YEAR,u.birthday,CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
    WHEN TIMESTAMPDIFF(YEAR,u.birthday,CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
    WHEN TIMESTAMPDIFF(YEAR,u.birthday,CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
    WHEN TIMESTAMPDIFF(YEAR,u.birthday,CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
    ELSE '65 and over'
  END AS age_range,
  CASE
    WHEN u.birthday IS NOT NULL THEN TIMESTAMPDIFF(YEAR,u.birthday,CURRENT_DATE())
    ELSE 9999
  END AS patient_current_age,
  u.gender AS patient_gender,
  CASE
    WHEN u.gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
    WHEN u.gender IN ('M','Male','Male (including trans man)') THEN 'Male'
    WHEN u.gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
    ELSE 'Unknown'
  END AS patient_gender_code_conformed,
  1 AS z_record_is_active,
  'MPB001' AS z_src_system_instance,
  'MPB' AS z_src_system_id,
  DATE_FORMAT(u.created_at,'dd/MM/yyyy') AS z_src_system_created_date_time,
  u.creator_id AS z_src_system_created_by_user,
  DATE_FORMAT(u.updated_at,'dd/MM/yyyy') AS z_src_system_modified_date_time,
  u.updated_by AS z_src_system_modified_by_user,
  
  CURRENT_TIMESTAMP() AS z_record_created_date_time,
  CURRENT_USER() AS z_record_created_by_user,
  CURRENT_TIMESTAMP() AS z_record_modified_date_time,
  CURRENT_USER() AS z_record_modified_by_user
FROM (SELECT * FROM silver_drj_users WHERE profile_type='user' AND inactive=0) u;


Analysis of SESSION entity indicates 55 attributes in scope as per Monday board.
Current silver_sessions UDM table contains 50 columns.
Several attributes marked Ready for UAT are delivered.
Identified gaps primarily fall into:

Conformed/RDM-based attributes

Business logic derived flags

Additional organisational identifiers

Purpose-related attributes
Naming inconsistencies observed for certain flags which require alignment.

Monday Name	silver_sessions Name
session_is_clinical_session	session_is_clinical
session_interpreter_present	interpreter_present

✅ SESSION – Delivered Attributes (silver_sessions मध्ये आहेत)

हे attributes silver_sessions table मध्ये physically आहेत:

🔹 Core Identifiers

session_id

src_session_id

session_patient_id

session_care_professional_id

session_care_prof_hr_staff_number

session_care_prof_session_id

session_care_epi_id

session_contr_id

session_bu_id

z_src_system_id

z_src_system_instance

🔹 Date / Time

session_date_conformed

session_time_conformed

session_start_date_time

session_end_date_time

🔹 Duration

session_duration_mins

session_clinical_duration_mins

session_admin_duration_mins

session_derived_duration_mins

🔹 Status

session_status

session_status_conformed

session_attendance_did_not_happen

session_attended_flag

session_deleted_flag

session_bkd_in_future

🔹 Financial

session_charge

session_cost

🔹 Delivery / Product

session_delivery_method

session_cprod_src_id

session_cprod_name_conformed

session_cprod_product_conformed

session_cprod_service_conformed

session_cprod_mstr_service_conformed

session_provided_internal_external

🔹 Location

session_care_location_name

session_care_location_postcode

🔹 Clinical Flags

session_is_clinical

session_is_vhg_contract_session

interpreter_present

🔹 Audit

z_record_created_date_time

z_record_created_by_user

z_record_modified_date_time

z_record_modified_by_user

z_src_system_modified_date_time

z_src_system_modified_by_user

❌ SESSION – Gap / Not Delivered (Monday मध्ये आहेत पण silver_sessions मध्ये नाहीत)

हे attributes तुझ्या Monday board मध्ये दिसत होते पण silver_sessions मध्ये नाहीत:

🔸 Conformed / RDM Related

session_delivery_method_conformed

session_cancellation_reason_conformed

session_delivery_1to1_or_group_conformed

session_cancelled_at_short_notice_conformed

🔸 Business Logic Flags

session_is_treatment_session

session_is_utilisation_time

session_is_assessment_session

session_is_supervision_session

session_care_professional_is_supervisor

session_care_professional_is_supervisee

🔸 Additional Identifiers

session_service_provider_org_id

session_sub_contract_id

session_care_location_id

session_language_used_id

🔸 Financial Variant

session_std_charge

session_cancelled_chargeable

🔸 Purpose Related

session_purpose

session_purpose_conformed